In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.model_selection import cross_val_score


clean_data= pd.read_csv('model_data.csv')
print(clean_data.columns)


Index(['Datum', 'Umsatz', 'Wettercode', 'KielerWoche', 'Jahreszeit_1',
       'Jahreszeit_2', 'Jahreszeit_3', 'Jahreszeit_4', 'Wetter_Kalt',
       'Wetter_Mild', 'Wetter_Warm', 'Feiertag', 'KielLauf',
       'Kieler_Triathlon', 'Fußball', 'PaycheckEffect', 'Sensitivity_High',
       'Sensitivity_Moderate', 'Wind_Nicht windig', 'Wind_Windig',
       'Wind_Zu windig', 'Inflation_Kategorisierung_Neutral',
       'Inflation_Kategorisierung_Positiv',
       'Inflation_Kategorisierung_vormonat_Neutral',
       'Inflation_Kategorisierung_vormonat_Positiv', 'gefühl_Kalt',
       'gefühl_Mild', 'gefühl_Warm', 'Montag', 'Dienstag', 'Mittwoch',
       'Donnerstag', 'Freitag', 'Samstag', 'Sonntag', 'Brot', 'Brötchen',
       'Croissant', 'Konditorei', 'Kuchen', 'Saisonbrot', 'Wochenende',
       'Ferien'],
      dtype='object')


In [42]:

# Features und Ziel definieren
# Beispiel: Ziel ist "Umsatz", Features sind andere numerische Spalten
feature_to_drop= ['Datum', 'Umsatz', 'Wettercode',   'KielLauf','Kieler_Triathlon', 'Fußball', 
       'Wind_Nicht windig', 'Wind_Windig','Wind_Zu windig', 'Inflation_Kategorisierung_Neutral',
       'Inflation_Kategorisierung_Positiv','Inflation_Kategorisierung_vormonat_Neutral',
       'Inflation_Kategorisierung_vormonat_Positiv',   'Montag', 'Dienstag', 'Mittwoch',
       'Donnerstag', 'Freitag', 'Samstag', 'Sonntag']

train = clean_data[clean_data['Datum'] <= '2017-12-31']
test = clean_data[clean_data['Datum'] > '2017-12-31']



X_train = train.drop(columns=feature_to_drop)  # Features: Alle außer 'Umsatz' und 'Datum'

y_train = train['Umsatz']  # Ziel: Umsatz

X_test= test.drop(columns=feature_to_drop)

y_test = test['Umsatz']


# Train-Test-Split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Lineares Regressionsmodell trainieren
model = LinearRegression()
model.fit(X_train, y_train)

# Vorhersagen
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)


print(y_pred_test)
# Modellbewertung
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

# Ergebnisse ausgeben
print(f"Trainingsdaten: MSE = {mse_train:.2f}, R² = {r2_train:.2f}")
print(f"Testdaten: MSE = {mse_test:.2f}, R² = {r2_test:.2f}")
print("Anzahl der Features:", len(X_train.columns))
print("Anzahl der Koeffizienten:", len(model.coef_))
# Wichtige Koeffizienten und Features
coefficients = pd.DataFrame({'Feature': X_train.columns, 'Coefficient': model.coef_})
print("\nKoeffizienten der Features:")
print(coefficients.sort_values(by='Coefficient', ascending=False))

scores = cross_val_score(model, X_train, y_train, cv=30, scoring='r2')
print(scores)

[119.875   119.875   119.875   114.375   162.53125  71.4375   71.4375
  71.4375   71.4375   71.4375  124.53125 124.53125  71.4375   71.4375
  71.4375   71.4375   71.4375  124.53125 108.46875  71.4375   71.4375
  71.34375  76.875    76.875   134.96875 134.96875  81.875    81.875
  81.875    76.375    76.375   113.40625 129.46875  60.3125   55.375
  55.375    55.375    55.375   124.53125 124.53125  71.4375   71.4375
  71.4375   71.4375   71.4375  124.53125 124.53125  71.4375   71.4375
  71.4375   71.4375   71.4375  124.53125 108.46875  60.3125   60.3125
  74.625    74.625   127.71875 127.71875  80.125    75.1875   69.6875
  69.6875   69.6875  122.78125 144.3125   75.1875   69.6875   69.6875
  69.6875  122.78125 122.78125  69.6875   69.6875   69.6875   75.1875
  69.6875  128.28125 128.28125  75.1875   80.125    74.625   112.625
 165.71875 181.375   134.21875 134.21875 118.125   113.1875  182.3125
 188.3125  140.75    113.1875  113.1875  113.1875  128.28125 128.28125
  91.21875 102.75     

In [17]:
import pandas as pd

# Beispiel-Datensatz laden
data = pd.read_csv('model_data.csv')  # Datensatz laden

# Konvertiert die Datumswerte (falls sie nicht schon datetime-Format haben)
#data['Datum      '] = pd.to_datetime(data['Datum      '], format='%Y-%m-%d')
# Trainingsdatensatz: 01.07.2013 bis 31.07.2017
train_data = data[(data['Datum'] >= '2013-07-01') & (data['Datum'] <= '2017-07-31')]

# Validierungsdatensatz: 01.08.2017 bis 31.07.2018
validation_data = data[(data['Datum'] >= '2017-08-01') & (data['Datum'] <= '2019-07-31')]

# Beispiel: Ziel ist "Umsatz"
X_train = train_data.drop(columns=['Datum','Umsatz','Wettercode'])
y_train = train_data['Datum']

X_validation = validation_data.drop(columns=['Datum','Umsatz','Wettercode'])
y_validation = validation_data['Datum']

from sklearn.metrics import mean_squared_error, r2_score

# Baseline-Modell: Durchschnitt als Vorhersage
baseline_prediction = y_train.mean()

# Vorhersagen für den Validierungsdatensatz
y_pred_baseline = [baseline_prediction] * len(y_validation)

# Bewertung des Baseline-Modells
mse = mean_squared_error(y_validation, y_pred_baseline)
r2 = r2_score(y_validation, y_pred_baseline)

print(f"Baseline MSE: {mse:.2f}")
print(f"Baseline R²: {r2:.2f}")

TypeError: Could not convert string '2013-07-012013-07-032013-07-042013-07-082013-07-102013-07-112013-07-142013-07-182013-07-192013-07-282013-07-302013-07-312013-08-012013-08-032013-08-082013-08-092013-08-102013-08-112013-08-122013-08-132013-08-142013-08-162013-08-172013-08-182013-08-202013-08-242013-08-252013-08-262013-08-302013-08-312013-09-012013-09-022013-09-032013-09-082013-09-092013-09-102013-09-112013-09-122013-09-132013-09-142013-09-152013-09-162013-09-172013-09-192013-09-222013-09-232013-09-252013-09-272013-10-012013-10-032013-10-042013-10-052013-10-082013-10-092013-10-112013-10-122013-10-152013-10-172013-10-202013-10-212013-10-242013-10-252013-10-262013-10-272013-10-282013-10-292013-10-302013-10-312013-11-012013-11-022013-11-032013-11-042013-11-052013-11-062013-11-072013-11-092013-11-102013-11-122013-11-132013-11-142013-11-152013-11-172013-11-182013-11-192013-11-212013-11-222013-11-272013-11-292013-11-302013-12-012013-12-022013-12-032013-12-042013-12-052013-12-062013-12-07201